## 네이버 금융에서 환율 정보 추출하기

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req

url = 'https://finance.naver.com/marketindex/'
res = req.urlopen(url)

soup = BeautifulSoup(res, 'html.parser')

price = soup.select_one('div.head_info > span.value').string
print('usd/krw', price)

usd/krw 1,189.50


## 위키 문헌에 공개돼 있는 이상 작가 작품 목록 가져오기

In [2]:
from bs4 import BeautifulSoup
import urllib.request as req

url = 'https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9D%B4%EC%83%81'
res = req.urlopen(url)
soup = BeautifulSoup(res, 'html.parser')

a_list = soup.select('#mw-content-text > div > ul > li > a')

for a in a_list:
    name = a.string
    print('-', name)

- 12월 12일
- 휴업과 사정
- 지도의 암실
- 지팽이 역사
- 지주회시
- 날개
- 동해
- 봉별기
- 황소와 도깨비
- 공포의 기록
- 종생기
- 환시기
- 단발
- 실화
- 김유정론
- 조감도
- 삼차각설계도
- 건축무한육면각체
- 오감도
- 역단
- 위독
- 이상한 가역반응
- 파편의 경치
- ▽의 유희
- 수염
- BOITEUX BOITEUSE
- 공복
- 꽃나무
- 이런 시
- 1933, 6, 1
- 거울
- 보통기념
- 소영위제
- 정식
- 지비
- 지비
- 가외가전
- 명경
- 아침
- 1931년
- 최후
- 내과
- 육친의 장
- 회한의 장
- 골편에 관한 무제
- I WED A TOY BRIDE
- 파첩
- 혈서삼태
- 산촌여정
- 서망율도
- EPIGRAM
- 약수
- 행복
- 19세기식
- 추등잡필
- 동생 옥희 보아라
- 권태
- 슬픈 이야기
- 문학과 정치
- 병상 이후
- 동경
- 최저낙원
- 실낙원
- 아름다운 조선말
- 시와 소설 편집후기
- 편지


## 모든 페이지를 한꺼번에 다운받는 프로그램

In [3]:
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re

# 이미 처리한 파일인지 확인하기 위한 변수
proc_files = {}

# HTML 내부에 있는 링크를 추출하는 함수
def enum_links(html, base):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select("link[rel='stylesheet']") # CSS
    links += soup.select("a[href]")# 링크
    result = []
    # href 속성을 추출하고, 링크를 절대 경로로 변환해야함
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# 파일을 다운받고 저장하는 함수
def download_file(url):
    o = urlparse(url)
    savepath = "./"+o.netloc+o.path
    if re.search(r"/$", savepath) : # 폴더라면 index.html
        savepath+='index.html'
    savedir = os.path.dirname(savepath)
    # 모두 다운됐는지 확인
    if os.path.exists(savepath): return savepath
    # 다운받을 폴더 생성
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # 파일 다운받기
    try :
        print("download=",url)
        urlretrieve(url, savepath)
        time.sleep(1)
        return savepath
    except:
        print('다운 실패', url)
        return None


# HTML을 분석하고 다운받는 함수
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None : return
    if savepath in proc_files : return # 이미 처리됐다면 실행하지 않음
    proc_files[savepath] = True
    print("analyze_html", url)
    # 링크추출
    html = open(savepath, "r", encoding = "utf-8").read()
    links = enum_links(html, url)
    
    for link_url in links:
        # 링크가 루트 이외의 경로를 나타낸다면 무시
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url) : continue
        # HTML 이라면
        if re.search(r".(html|htm)$", link_url):
            analyze_html(link_url, root_url)
            continue
        # 기타 파일
        download_file(link_url)

if __name__ == "__main__":
    # url에 있는 모든 것 다운받기
    url = "https://docs.python.org/3.8/library"
    analyze_html(url, url)

mkdir= ./docs.python.org/3.8
download= https://docs.python.org/3.8/library
analyze_html https://docs.python.org/3.8/library
mkdir= ./docs.python.org/_static
download= https://docs.python.org/_static/pydoctheme.css
download= https://docs.python.org/_static/pygments.css
